In [1]:
import sys
sys.path.append('../')


In [2]:
import pnn

In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [4]:
def gen_pattern_replace_and_matcher_for_loop_pnn(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]

      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

    def pn(self,reducey_x,unredency_x):
       redency_x_row, reducey_x_col = list(),list()
       unredency_x_row, unredency_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redency_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             unredency_x_row.append(i),unredency_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redency_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = 2),\
              torch.sum(unredency_x[:,unredency_x_row] * unredency_x[:,unredency_x_col],dim = 2),\
              torch.sum(reducey_x[mixed_x_row] * unredency_x[:,mixed_x_col],dim = 2)


    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_part_embed = self.embed(redency_part)
      unredency_part_embed = self.embed(unredency_part)
      redency_part_pn, unredency_part_pn, mixed_part_pn = self.pn(redency_part_embed,unredency_part_embed)
      
      
      return self.redency_linear(self.embed(redency_part).view(-1,self.embed_dim * self.num_prefix)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.embed_dim * self.num_sufix)) +\
         self.redency_cross_part_linear(redency_part_pn) + self.mixed_cross_part_linear(mixed_part_pn) + self.unredency_cross_part_linear(unredency_part_pn)
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

# pnn_loop

In [6]:
def workload_pnn(dim):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [5]:
pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(100)],32,[400,400,400],0.1)


In [6]:
interp = utils.ProfilingInterpreter(pnn_loop)
interp.run(torch.randint(low=0, high=88, size=(4096,100), dtype=torch.long))
print(interp.summary(True))

total true time 393.45717430114746 ms
total time: 523.1425762176514 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  mul                             108.499              20.7399
call_function  getitem                         108.199              20.6824
call_function  getitem_1                       105.539              20.1741
call_function  sum_1                            26.2599              5.01964
call_module    mlp_mlp_0                        13.1128              2.50654
call_function  cat                              10.9351              2.09027
call_module    embedding_embedding               6.72746             1.28597
call_function  add                               6.47187             1.23711
call_module    mlp_mlp_2                         1.29151             0.246876
call_module    mlp_mlp_6                         1.12271             0.214609


# dim = 8

In [182]:
ori , modify = workload_pnn(8)

now gen workload of pnn with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [183]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


3.41 ms ± 382 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [186]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

13.7 ms ± 2.24 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [188]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


7.36 ms ± 721 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [190]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

17.9 ms ± 2.14 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [181]:
%timeit -n 1 -r 300 interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

4.13 ms ± 560 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [170]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 7.602930068969727 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                           1.00636              13.2365
call_function  getitem_1                         0.874996             11.5087
call_module    mlp_mlp_0                         0.7267                9.55815
call_module    mlp_mlp_3                         0.564814              7.42889
call_module    mlp_mlp_6                         0.563145              7.40694
call_module    embedding_embedding               0.418663              5.5066
call_module    mlp_mlp_8                         0.398636              5.24319
call_module    mlp_mlp_5                         0.392199              5.15852
call_module    mlp_mlp_2                         0.388622              5.11148
call_function  sum_1                             0.375509              4.93901
call_function  mul  

In [21]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 18.455982208251953 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_module    embed_3                                   5.66602             30.7002
call_function  add_3                                     4.0853              22.1354
call_module    mlp_mlp_2                                 1.59907              8.66426
call_module    mlp_mlp_3                                 0.517845             2.80584
call_module    mlp_mlp_6                                 0.493526             2.67407
call_function  getitem_7                                 0.418186             2.26586
call_module    mlp_mlp_8                                 0.346184             1.87573
call_module    unredency_linear                          0.334978             1.81501
call_module    mlp_mlp_5                                 0.331879             1.79822
call_module    mix

In [27]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 14.519214630126953 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                           2.39992              16.5293
call_function  getitem_1                         2.16126              14.8855
call_module    mlp_mlp_3                         1.29604               8.9264
call_module    mlp_mlp_0                         1.24526               8.57664
call_module    mlp_mlp_6                         1.16539               8.02654
call_function  sum_1                             1.11318               7.66692
call_function  mul                               1.08528               7.47479
call_module    mlp_mlp_2                         0.638008              4.39423
call_module    mlp_mlp_5                         0.480413              3.30881
call_function  cat                               0.419855              2.89172
call_module    mlp_

In [36]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 22.56155014038086 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_module    embed_3                                  4.04572              17.9319
call_function  add_3                                    3.66139              16.2285
call_function  getitem_7                                1.24502               5.51833
call_function  add_2                                    1.19209               5.28374
call_module    mlp_mlp_3                                1.10292               4.88851
call_module    mlp_mlp_6                                0.964642              4.2756
call_function  getitem_4                                0.791073              3.50629
call_function  getitem_5                                0.733852              3.25267
call_function  sum_3                                    0.656605              2.91028
call_module    mlp_m

# dim = 32

In [191]:
ori , modify = workload_pnn(32)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [192]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


5.41 ms ± 572 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [194]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

11.5 ms ± 2.76 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [196]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


52 ms ± 1.85 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [198]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

38.3 ms ± 2.91 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [41]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 10.355234146118164 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  mul                               1.18494              11.4429
call_function  getitem                           1.15371              11.1413
call_function  getitem_1                         1.12987              10.9111
call_module    mlp_mlp_0                         1.04523              10.0937
call_module    mlp_mlp_2                         0.843525              8.14588
call_module    embedding_embedding               0.743628              7.18118
call_module    mlp_mlp_3                         0.570774              5.51194
call_module    mlp_mlp_6                         0.528812              5.10672
call_function  cat                               0.424862              4.10287
call_function  sum_1                             0.37837               3.6539
call_module    mlp_ml

In [47]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 14.546632766723633 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  add_3                                     5.13268             35.2843
call_function  getitem_7                                 0.767469             5.27592
call_module    mlp_mlp_2                                 0.680685             4.67933
call_module    mlp_mlp_3                                 0.558138             3.83689
call_module    mlp_mlp_6                                 0.527382             3.62546
call_module    unredency_linear                          0.52309              3.59595
call_function  getitem_5                                 0.486374             3.34355
call_function  getitem_4                                 0.47183              3.24357
call_module    embed_3                                   0.422001             2.90102
call_module    ml

In [73]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 40.73166847229004 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                          11.076               27.1925
call_function  getitem_1                         7.83324             19.2313
call_function  mul                               6.74272             16.554
call_module    mlp_mlp_0                         1.76692              4.33795
call_function  sum_1                             1.59216              3.9089
call_module    mlp_mlp_3                         0.901937             2.21434
call_module    mlp_mlp_6                         0.832319             2.04342
call_function  cat                               0.817299             2.00654
call_module    embedding_embedding               0.483513             1.18707
call_module    mlp_mlp_2                         0.467062             1.14668
call_module    mlp_mlp_5        

In [61]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 33.04886817932129 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  getitem_7                                6.33836              19.1787
call_function  mul_2                                    4.01521              12.1493
call_function  add_3                                    3.54218              10.718
call_function  mul_1                                    1.77193               5.36154
call_function  getitem_5                                1.53589               4.64734
call_function  getitem_4                                1.5161                4.58746
call_function  sum_3                                    0.950813              2.87699
call_module    mlp_mlp_3                                0.924826              2.79836
call_module    mlp_mlp_6                                0.832558              2.51917
call_module    unrede

# dim = 64

In [199]:
ori , modify = workload_pnn(64)

now gen workload of pnn with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [202]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


27.2 ms ± 1.09 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [203]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

15.1 ms ± 2.52 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [205]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


95.4 ms ± 2.56 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [206]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

71.4 ms ± 3.64 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [79]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 23.23627471923828 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem_1                         5.61929             24.1833
call_function  mul                               4.14872             17.8545
call_function  getitem                           2.47383             10.6464
call_module    mlp_mlp_0                         1.26338              5.4371
call_function  sum_1                             0.88048              3.78925
call_function  cat                               0.427723             1.84076
call_module    mlp_mlp_3                         0.393629             1.69403
call_module    mlp_mlp_6                         0.384092             1.65299
call_module    mlp_mlp_2                         0.364065             1.5668
call_module    embedding_embedding               0.354528             1.52575
call_module    mlp_mlp_5        

In [86]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 25.43473243713379 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  add_3                                    10.6308              41.7966
call_module    mlp_mlp_8                                 2.15697              8.48042
call_function  getitem_7                                 1.24216              4.88372
call_function  mul_2                                     1.20735              4.74686
call_function  mul_1                                     1.18256              4.64938
call_module    unredency_linear                          0.902414             3.54796
call_function  getitem_4                                 0.838041             3.29487
call_function  getitem_5                                 0.752926             2.96023
call_function  sum_3                                     0.565052             2.22158
call_module    mlp

In [90]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 64.21279907226562 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                          17.2462              26.858
call_function  getitem_1                        12.6898              19.7621
call_function  mul                              12.6648              19.7232
call_module    mlp_mlp_0                         2.64978              4.12657
call_function  sum_1                             2.52676              3.93498
call_function  cat                               1.10102              1.71464
call_module    mlp_mlp_3                         0.869513             1.35411
call_module    mlp_mlp_6                         0.779867             1.2145
call_module    embedding_embedding               0.698328             1.08752
call_module    mlp_mlp_2                         0.44179              0.688009
call_module    mlp_mlp_8       

In [102]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 62.342166900634766 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  add_3                                    7.11513              11.413
call_function  getitem_5                                6.60419              10.5935
call_function  mul_2                                    6.11377               9.80679
call_function  getitem_7                                5.99051               9.60907
call_function  mul_1                                    4.01926               6.4471
call_function  getitem_4                                2.41399               3.87216
call_function  sum_3                                    1.51157               2.42464
call_module    mlp_mlp_4                                1.26696               2.03226
call_module    unredency_linear                         1.24073               1.99019
call_module    mlp_m

# dim = 128

In [225]:
ori , modify = workload_pnn(128)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [208]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))


48.9 ms ± 2.63 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [210]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

39 ms ± 2.81 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [211]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))


168 ms ± 6.64 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [212]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

123 ms ± 5.71 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [107]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 37.18709945678711 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                          10.3178              27.7456
call_function  getitem_1                         7.06649             19.0025
call_function  mul                               6.73842             18.1203
call_module    mlp_mlp_0                         1.73831              4.6745
call_function  sum_1                             1.41311              3.79999
call_function  cat                               0.752449             2.02341
call_module    embedding_embedding               0.560999             1.50858
call_module    mlp_mlp_3                         0.368595             0.991191
call_module    mlp_mlp_2                         0.347614             0.934771
call_module    mlp_mlp_6                         0.333786             0.897585
call_module    mlp_mlp_8    

In [118]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
print(interp.summary(True))

total time: 37.3225212097168 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  add_3                                     5.60379             15.0145
call_function  getitem_7                                 5.09644             13.6551
call_function  mul_2                                     4.52209             12.1162
call_function  mul_1                                     3.90983             10.4758
call_function  getitem_4                                 1.40023              3.75171
call_function  getitem_5                                 1.30486              3.49619
call_module    unredency_linear                          0.992775             2.65999
call_function  sum_3                                     0.803471             2.15278
call_function  sum_2                                     0.455141             1.21948
call_module    mlp_mlp

In [124]:
interp = utils.ProfilingInterpreter(ori)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 107.4821949005127 ms
Op type        Op                     Average runtime (ms)    Pct total runtime
-------------  -------------------  ----------------------  -------------------
call_function  getitem                          27.5331              25.6164
call_function  mul                              22.6054              21.0318
call_function  getitem_1                        22.4383              20.8763
call_module    mlp_mlp_0                         4.6823               4.35635
call_function  sum_1                             4.36759              4.06355
call_function  cat                               2.58398              2.4041
call_module    embedding_embedding               0.943184             0.877526
call_module    mlp_mlp_3                         0.921965             0.857784
call_module    mlp_mlp_6                         0.840187             0.781699
call_module    mlp_mlp_2                         0.53215              0.495106
call_module    mlp_mlp_5   

In [130]:
interp = utils.ProfilingInterpreter(modify)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 92.64087677001953 ms
Op type        Op                             Average runtime (ms)    Pct total runtime
-------------  ---------------------------  ----------------------  -------------------
call_function  getitem_4                                13.401               14.4656
call_function  mul_2                                    12.0983              13.0594
call_function  getitem_7                                12.0935              13.0542
call_function  getitem_5                                 7.90715              8.53527
call_function  mul_1                                     7.48277              8.07718
call_function  add_3                                     7.11393              7.67904
call_function  sum_3                                     2.48909              2.68682
call_module    unredency_linear                          2.17676              2.34968
call_function  sum_2                                     1.53804              1.66022
call_module    mlp_m

# TorchScript

In [131]:
def workload_pnn_torchscript(dim):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  ori = torch.jit.script(pnn_model_traced_ori)
  modify = torch.jit.script(pnn_model_traced_modify)
  ori.eval()
  modify.eval()
  return ori,modify

# dim = 8

In [213]:
ori, modify = workload_pnn_torchscript(8)

now gen workload of pnn with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [214]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

The slowest run took 7.21 times longer than the fastest. This could mean that an intermediate result is being cached.
2.44 ms ± 907 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [216]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

5.86 ms ± 697 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [217]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

7.5 ms ± 918 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [218]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

9.6 ms ± 731 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 32

In [219]:
ori, modify = workload_pnn_torchscript(32)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [220]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

13.6 ms ± 612 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [222]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

5.85 ms ± 586 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [223]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

47.1 ms ± 2.04 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [224]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

36 ms ± 1.37 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 64

In [140]:
ori, modify = workload_pnn_torchscript(64)

now gen workload of pnn with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [142]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

24.7 ms ± 1.5 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [144]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

12.8 ms ± 570 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [145]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

75.5 ms ± 5.69 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [146]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

50.3 ms ± 4.12 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 128

In [147]:
ori, modify = workload_pnn_torchscript(128)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [151]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

46.5 ms ± 2.76 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [150]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

42.3 ms ± 2.79 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [152]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

162 ms ± 5.44 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [153]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

120 ms ± 6.95 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [228]:
ori, modify = workload_pnn(128)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [229]:
ori_1024 = torch.jit.script(ori,torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
ori_4096 = torch.jit.script(ori,torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_script.py:1291: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


In [235]:
type(ori_1024)

torch.jit._script.RecursiveScriptModule

In [236]:
type(ori)

torch.fx.graph_module.GraphModule.__new__.<locals>.GraphModuleImpl

In [231]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori_1024(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

53.5 ms ± 2.62 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [232]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori_4096(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

180 ms ± 6.66 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [237]:
ori_1024 = torch.jit.trace(ori,torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
ori_4096 = torch.jit.trace(ori,torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1017 / 1024 (99.3%)
Greatest absolute difference: 0.0068597495555877686 at index (728,) (up to 1e-05 allowed)
Greatest relative difference: 0.013722593341506177 at index (728,) (up to 1e-05 allowed)
  _check_trace(
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4058 / 4096 (99.1%)
Greatest absolute difference: 0.006524980068206787 at index (2157,) (up to 1e-05 allowed)
Greatest relative difference: 0.013065513636967988 at index (2157,) (up to 1e-05 allowed)
  _check_trace(


In [238]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori_1024(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

48.8 ms ± 1.78 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [239]:
%timeit -n 1 -r 300 with torch.no_grad(): output = ori_4096(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

172 ms ± 2.96 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [230]:
modify_1024  = torch.jit.script(modify,torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))
modify_4096  = torch.jit.script(modify,torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

In [ ]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify_1024(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

43.5 ms ± 3.81 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [234]:
%timeit -n 1 -r 300 with torch.no_grad(): output = modify_4096(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

118 ms ± 5.95 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [157]:
pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],128,[400,400,400],0.1)
pnn_loop_traced = symbolic_trace(pnn_loop)

In [163]:
interp = utils.ProfilingInterpreter(pnn_loop_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 118.07584762573242 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  mul                            0.0299106             25.3317
call_function  getitem                        0.0293577             24.8634
call_function  getitem_1                      0.0225163             19.0693
call_module    mlp_mlp_0                      0.00482535             4.08666
call_function  sum_1                          0.00405502             3.43425
call_function  cat                            0.00285244             2.41577
call_module    embedding_embedding            0.000957251            0.810708
call_module    mlp_mlp_3                      0.000898361            0.760834
call_module    mlp_mlp_6                      0.00078249             0.662701
call_module    mlp_mlp_2                      0.000460863            0.390311
call_module    mlp_mlp_5             

In [31]:
pnn_loop_traced.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embedding_offsets = self.embedding.offsets
        add = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add);  add = None
        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13,

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embedding_offsets = self.embedding.offsets\n        add = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add);  add = None\n        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 

In [32]:
pnn_loop_traced.graph.print_tabular()

opcode         name                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [164]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_loop_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)

In [25]:
pnn_loop_traced.graph.print_tabular()

opcode         name                 target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [165]:
matches = subgraph_rewriter.replace_pattern_with_filters(pnn_loop_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [35]:
pnn_loop_traced.graph.print_tabular()

opcode         name                         target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                kwargs
-------------  ---------------------------  ----------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [174]:
interp = utils.ProfilingInterpreter(pnn_loop_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 96.6329574584961 ms
Op type        Op                             Average runtime (s)    Pct total runtime
-------------  ---------------------------  ---------------------  -------------------
call_function  add_3                                  0.0126367            13.077
call_function  getitem_7                              0.0120206            12.4394
call_function  mul_2                                  0.0116725            12.0792
call_function  getitem_5                              0.0093472             9.67289
call_function  mul_1                                  0.00742221            7.68083
call_function  getitem_4                              0.00576401            5.96485
call_function  sum_3                                  0.00234294            2.42458
call_module    unredency_linear                       0.00216556            2.24101
call_module    mixed_cross_part_linear                0.0015583             1.6126
call_function  sum_2                       

In [21]:
type(replace)

__main__.gen_pattern_replace_and_matcher_for_loop_pnn.<locals>.ReplacementClass

In [187]:
pnn_v2 = pnn.ProductNeuralNetworkModel([100 for i in range(22)],128,[400,400,400],0.1, method='innerV2')

In [188]:
pnn_v2_traced = symbolic_trace(pnn_v2)

In [189]:
pnn_v2_traced.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embedding_offsets = self.embedding.offsets
        add = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add);  add = None
        transpose = embedding_embedding.transpose(1, 2)
        bmm = torch.bmm(embedding_embedding, transpose);  transpose = None
        pn_triu_mask = self.pn.triu_mask
        masked_select = torch.masked_select(bmm, pn_triu_mask);  bmm = pn_triu_mask = None
        view = masked_select.view(-1, 231);  masked_select = None
        view_1 = embedding_embedding.view(-1, 2816);  embedding_embedding = None
        cat = torch.cat([view_1, view], dim = 1);  view_1 = view = None
        mlp_mlp_0 = getattr(self.mlp.mlp, "0")(cat);  cat = None
        mlp_mlp_1 = getattr(self.mlp.mlp, "1")(mlp_mlp_0);  mlp_mlp_0 = None
        mlp_mlp_2 = getattr(self.mlp.mlp, "2")(mlp_mlp_1); 

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embedding_offsets = self.embedding.offsets\n        add = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add);  add = None\n        transpose = embedding_embedding.transpose(1, 2)\n        bmm = torch.bmm(embedding_embedding, transpose);  transpose = None\n        pn_triu_mask = self.pn.triu_mask\n        masked_select = torch.masked_select(bmm, pn_triu_mask);  bmm = pn_triu_mask = None\n        view = masked_select.view(-1, 231);  masked_select = None\n        view_1 = embedding_embedding.view(-1, 2816);  embedding_embedding = None\n        cat = torch.cat([view_1, view], dim = 1);  view_1 = view = None\n        mlp_mlp_0 = getattr(self.mlp.mlp, "0")(cat);  cat = None\n        mlp_mlp_1 = getattr(self.mlp.mlp, "1")(mlp_mlp_0);  mlp_mlp_0 = None\n        mlp_mlp_2 = getattr(self.mlp.mlp, "

In [193]:
interp = utils.ProfilingInterpreter(pnn_v2_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 24.57261085510254 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  masked_select                  0.00524545            21.3467
call_module    mlp_mlp_0                      0.00513387            20.8926
call_function  cat                            0.0046227             18.8124
call_function  bmm                            0.0023315              9.48819
call_module    embedding_embedding            0.00134754             5.48392
call_module    mlp_mlp_3                      0.00107288             4.36618
call_module    mlp_mlp_6                      0.000883102            3.59385
call_module    mlp_mlp_2                      0.000812531            3.30665
call_module    mlp_mlp_8                      0.000488043            1.98613
call_module    mlp_mlp_5                      0.000458479            1.86581
call_module    mlp_mlp_1                  

In [ ]:
pnn_outer = pnn.ProductNeuralNetworkModel([100 for i in range(22)],8,[400,400,400],0.1, method='outer')
pnn_outer_traced = symbolic_trace(pnn_outer)

In [204]:
pnn_outer_traced.print_readable()

class ProductNeuralNetworkModel(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        embedding_offsets = self.embedding.offsets
        add = x + embedding_offsets;  x = embedding_offsets = None
        embedding_embedding = self.embedding.embedding(add);  add = None
        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13,

'class ProductNeuralNetworkModel(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        embedding_offsets = self.embedding.offsets\n        add = x + embedding_offsets;  x = embedding_offsets = None\n        embedding_embedding = self.embedding.embedding(add);  add = None\n        getitem = embedding_embedding[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 

In [224]:
pnn_outer_traced.graph.print_tabular()

opcode         name                         target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                kwargs
-------------  ---------------------------  ----------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [225]:
kernel = utils.get_target_mod(pnn_outer_traced,'kernel')

In [226]:
type(kernel)

torch.nn.parameter.Parameter

In [227]:
kernel.shape

torch.Size([8, 231, 8])

In [228]:
kernel[:,1,:]

tensor([[ 0.0184,  0.0312,  0.0200,  0.0395,  0.0076,  0.0135,  0.0318, -0.0034],
        [-0.0161, -0.0463, -0.0146, -0.0019, -0.0199, -0.0469,  0.0476,  0.0186],
        [-0.0297, -0.0293, -0.0439, -0.0461, -0.0386, -0.0406,  0.0221, -0.0119],
        [-0.0164,  0.0205, -0.0020, -0.0465,  0.0514, -0.0360,  0.0171, -0.0087],
        [ 0.0235,  0.0047,  0.0312, -0.0522,  0.0282,  0.0534, -0.0412,  0.0320],
        [-0.0088,  0.0326, -0.0248, -0.0116, -0.0090,  0.0271, -0.0495, -0.0105],
        [ 0.0008, -0.0024, -0.0204,  0.0263,  0.0297, -0.0052,  0.0176, -0.0262],
        [-0.0214, -0.0086, -0.0206, -0.0223,  0.0485,  0.0100,  0.0408,  0.0429]],
       grad_fn=<SliceBackward0>)

In [205]:
sample = torch.ones((4096,22), dtype=torch.long)


torch.onnx.export(pnn_outer,sample,f'pnn_outer.onnx')

In [206]:
from onnx.shape_inference import infer_shapes
from onnx import load_model, save_model
onnx_model = load_model(f'pnn_outer.onnx')
onnx_model = infer_shapes(onnx_model)

save_model(onnx_model, f'pnn_outer.onnx')

In [257]:
pnn_outer = pnn.ProductNeuralNetworkModel([100 for i in range(22)],8,[400,400,400],0.1, method='outer')
pnn_outer_traced = symbolic_trace(pnn_outer)

In [274]:
interp = utils.ProfilingInterpreter(pnn_outer_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 58.17294120788574 ms
Op type        Op                     Average runtime (s)    Pct total runtime
-------------  -------------------  ---------------------  -------------------
call_function  mul                            0.0169675             29.1674
call_function  sum_1                          0.00409532             7.0399
call_function  getitem_1                      0.00242662             4.1714
call_function  getitem                        0.0024116              4.14558
call_function  mul_1                          0.00127482             2.19144
call_module    mlp_mlp_0                      0.00123858             2.12914
call_module    mlp_mlp_6                      0.00113988             1.95947
call_module    mlp_mlp_3                      0.00112081             1.92668
call_function  sum_2                          0.000507116            0.871739
call_module    mlp_mlp_2                      0.000453949            0.780344
call_module    mlp_mlp_5                

In [209]:
class PatternClass(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = torch.nn.Embedding(1, 1)
        self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
        self.embed_output_dim = 176
        self.mlp = nn.Linear(1,1)
        self.num_fields = 22
        self.kernel = torch.nn.Parameter(torch.zeros((1,1,1)))

    def pn(self,x):
        num_fields = self.num_fields
        row, col = list(), list()
        for i in range(num_fields - 1):
            for j in range(i + 1, num_fields):
                row.append(i), col.append(j)
        p, q = x[:, row], x[:, col]
        kp = torch.sum(p.unsqueeze(1) * self.kernel, dim=-1).permute(0, 2, 1)
        return torch.sum(kp * q, -1)

    def forward(self,x):
        x = self.embed(x + self.offsets)
        cross_term = self.pn(x)
        x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
        return self.mlp(x)

In [210]:
pn = symbolic_trace(PatternClass())

In [211]:
pn.print_readable()

class PatternClass(torch.nn.Module):
    def forward(self, x):
        # No stacktrace found for following nodes
        offsets = self.offsets
        add = x + offsets;  x = offsets = None
        embed = self.embed(add);  add = None
        getitem = embed[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16

'class PatternClass(torch.nn.Module):\n    def forward(self, x):\n        # No stacktrace found for following nodes\n        offsets = self.offsets\n        add = x + offsets;  x = offsets = None\n        embed = self.embed(add);  add = None\n        getitem = embed[(slice(None, None, None), [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 16, 16,

In [215]:
pn.graph.print_tabular()

opcode         name       target                                                  args                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 k

In [213]:
matcher =  SubgraphMatcher(pn.graph, match_output=False, match_placeholder=False,
                            remove_overlapping_matches=True)
_matches = matcher.match(pnn_outer_traced.graph)

In [214]:
_matches

[InternalMatch(anchors=[mlp], nodes_map={mlp: mlp_mlp_0, cat: cat, view: view, embed: embedding_embedding, add: add, x: x, offsets: embedding_offsets, sum_2: sum_2, mul_1: mul_1, permute: permute, sum_1: sum_1, mul: mul, unsqueeze: unsqueeze, getitem: getitem, kernel: pn_kernel, getitem_1: getitem_1}, placeholder_nodes=[x], returning_nodes=[mlp_mlp_0], name_node_map={})]

In [240]:
def gen_pattern_replace_and_matcher_for_outer_pnn(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.offsets = torch.as_tensor(np.array((0, *np.cumsum([10,10])[:-1]), dtype=np.int64))
          self.embed_output_dim = shape_info[1] * num_field
          self.mlp = nn.Linear(shape_info[0],shape_info[1])
          self.kernel = torch.nn.Parameter(torch.zeros((1,1,1)))

      def pn(self,x):
         p , q = x[getitem_node_args[0]] , x[getitem_node_args[1]]
         kp = torch.sum(p.unsqueeze(1) * self.kernel, dim=-1).permute(0, 2, 1)
         return torch.sum(kp * q, -1)

      def forward(self,x):
          x = self.embed(x + self.offsets)
          cross_term = self.pn(x)
          x = torch.cat([x.view(-1, self.embed_output_dim), cross_term], dim=1)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  pn = pattern_env['offsets']
  offsets_node = node_map[pn]
  offsets_val = utils.get_target_mod(traced,offsets_node.target)
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  kernel_node = node_map[pattern_env['kernel']]
  kernel_node_module = utils.get_target_mod(traced,kernel_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.redency_offset = torch.as_tensor(np.array(offsets_val[redency_part_slice[1]], dtype=np.int64))
      self.unredency_offset = torch.as_tensor(np.array(offsets_val[unredency_part_slice[1]], dtype=np.int64))
      self.num_fields = num_field
      self.num_prefix = redency_part_slice[1].stop
      self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redency_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.unredency_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redency_cross_part_total - self.unredency_cross_part_total
      # 提取对应的权重参数逻辑有些复杂，先mock
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]

      # cross part linear
      self.redency_cross_part_linear = nn.Linear(self.redency_cross_part_total,self.ori_linear_shape[0],bias = True)
      self.unredency_cross_part_linear = nn.Linear(self.unredency_cross_part_total,self.ori_linear_shape[0],bias = False)
      self.mixed_cross_part_linear = nn.Linear(self.rest_cross_part_total,self.ori_linear_shape[0],bias = False)
      
      # embed part linear 
      self.redency_linear = nn.Linear(self.embed_dim * self.num_prefix,self.ori_linear_shape[0],bias = False)
      self.unredency_linear = nn.Linear(self.embed_dim * self.num_sufix,self.ori_linear_shape[0],bias = False)

      # kernel 同理，先mock
      
      self.kernel = kernel_node_module
      self.redency_kernel = torch.nn.Parameter(self.kernel[:,:self.redency_cross_part_total])
      self.unredency_kernel = torch.nn.Parameter(self.kernel[:,:self.unredency_cross_part_total])
      self.mixed_kernel = torch.nn.Parameter(self.kernel[:,:self.rest_cross_part_total])
      

    def pn(self,reducey_x,unredency_x):
      redency_x_row, reducey_x_col = list(),list()
      unredency_x_row, unredency_x_col = list(), list()
      mixed_x_row, mixed_x_col = list(), list()
      prefix = self.num_prefix
      sufix = self.num_sufix
      for i in range(prefix - 1):
         for j in range(i+1,prefix):
            redency_x_row.append(i),reducey_x_col.append(j)
      for i in range(sufix - 1):
         for j in range(i + 1,sufix):
            unredency_x_row.append(i),unredency_x_col.append(j)

      for i in range(prefix):
         for j in range(sufix):
            mixed_x_row.append(i),mixed_x_col.append(j)
      kp_redency = torch.sum(reducey_x[redency_x_row].unsqueeze(0).unsqueeze(1)* self.redency_kernel, dim=-1).permute(0, 2, 1)
      kp_unredency = torch.sum(unredency_x[:,unredency_x_row].unsqueeze(1)* self.unredency_kernel, dim=-1).permute(0, 2, 1)
      kp_mixed = torch.sum(reducey_x[mixed_x_row].unsqueeze(0).unsqueeze(1)* self.mixed_kernel, dim=-1).permute(0, 2, 1)
      return torch.sum((kp_redency * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
            torch.sum(kp_unredency * unredency_x[:,unredency_x_col],dim = -1),\
            torch.sum(kp_mixed * unredency_x[:,mixed_x_col],dim = -1)


    def forward(self,x):
      redency_part = x[redency_part_slice] + self.redency_offset
      unredency_part = x[unredency_part_slice] + self.unredency_offset
      redency_part_embed = self.embed(redency_part)
      unredency_part_embed = self.embed(unredency_part)
      redency_part_pn, unredency_part_pn, mixed_part_pn = self.pn(redency_part_embed,unredency_part_embed)
      
      
      return self.redency_linear(self.embed(redency_part).view(-1,self.embed_dim * self.num_prefix)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.embed_dim * self.num_sufix)) +\
         self.redency_cross_part_linear(redency_part_pn) + self.mixed_cross_part_linear(mixed_part_pn) + self.unredency_cross_part_linear(unredency_part_pn)
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [286]:
pnn_outer = pnn.ProductNeuralNetworkModel([100 for i in range(22)],8,[400,400,400],0.1, method='outer')
pnn_outer_traced = symbolic_trace(pnn_outer)

In [287]:
pattern,replace,match = gen_pattern_replace_and_matcher_for_outer_pnn(pnn_outer_traced,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)

In [288]:
matches = subgraph_rewriter.replace_pattern_with_filters(pnn_outer_traced, pattern, replace,[match])

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [220]:
matches

[ReplacedPatterns(anchor=mlp, nodes_map={mlp: mlp_mlp_0, cat: cat, view: view, embed: embedding_embedding, add: add, x: x, offsets: embedding_offsets, sum_2: sum_2, mul_1: mul_1, permute: permute, sum_1: sum_1, mul: mul, unsqueeze: unsqueeze, getitem: getitem, kernel: pn_kernel, getitem_1: getitem_1}, replacements=[getitem_2, redency_offset, add_1, getitem_3, unredency_offset, add_2, embed, embed_1, getitem_4, unsqueeze_1, unsqueeze_2, kernel, mul_2, sum_3, permute_1, getitem_5, unsqueeze_3, mul_3, sum_4, permute_2, getitem_6, unsqueeze_4, unsqueeze_5, mul_4, sum_5, permute_3, getitem_7, mul_5, unsqueeze_6, sum_6, getitem_8, mul_6, sum_7, getitem_9, mul_7, sum_8, embed_2, view_1, redency_linear, embed_3, view_2, unredency_linear, add_3, redency_cross_part_linear, add_4, mixed_cross_part_linear, add_5, unredency_cross_part_linear, add_6])]

In [222]:
pnn_outer_traced.graph.print_tabular()

opcode         name                         target                                                      args                                                                                                                                                                                                                                                                                                                                                                                                                                kwargs
-------------  ---------------------------  ----------------------------------------------------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [298]:
sample = torch.ones((4096,22), dtype=torch.long)
modify_mod = GraphModule(pnn_outer_traced, pnn_outer_traced.graph)

torch.onnx.export(modify_mod,sample,f'pnn_outer_modify.onnx')

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset11.py:958: UserWarning: This model contains a squeeze operation on dimension 1. The size of this dimension in the given input is 4096. The model will be exported without the squeeze node. If the model is intended to be used with dynamic input shapes, please export with dynamic_axes argument.
  warnings.warn(


In [297]:
interp = utils.ProfilingInterpreter(pnn_outer_traced)
interp.run(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))
print(interp.summary(True))

total time: 34.79623794555664 ms
Op type        Op                             Average runtime (s)    Pct total runtime
-------------  ---------------------------  ---------------------  -------------------
call_function  mul_1                                  0.00686979            19.7429
call_module    embed_3                                0.00499916            14.367
call_function  add_3                                  0.00240707             6.91763
call_function  sum_2                                  0.00161886             4.65241
call_module    mlp_mlp_3                              0.000922918            2.65235
call_module    mlp_mlp_6                              0.000880718            2.53107
call_function  getitem_3                              0.000858068            2.46598
call_function  getitem_7                              0.000759125            2.18163
call_module    unredency_linear                       0.000613451            1.76298
call_module    mixed_cross_part

In [241]:
def workload_pnn_outer(dim):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1, method='outer')

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1, method='outer')
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_outer_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

# dim = 8

In [285]:
ori, modify = workload_pnn_outer(8)

now gen workload of pnn with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [286]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

11.6 ms ± 916 µs per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [287]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

13 ms ± 1.48 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [245]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

42.8 ms ± 2.13 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [246]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

28 ms ± 2.65 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 32

In [247]:
ori, modify = workload_pnn_outer(32)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [248]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

114 ms ± 5.71 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [249]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

43.5 ms ± 3.79 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [250]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

446 ms ± 17.7 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [251]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

158 ms ± 7.34 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 64

In [252]:
ori, modify = workload_pnn_outer(64)

now gen workload of pnn with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [253]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

430 ms ± 17.5 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [254]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

137 ms ± 6.86 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [255]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

1.62 s ± 18.3 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [256]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

499 ms ± 18.7 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


# dim = 128

In [257]:
ori, modify = workload_pnn_outer(128)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '


In [258]:
%timeit -n 1 -r 300 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.56 s ± 28.1 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [259]:
%timeit -n 1 -r 300 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

462 ms ± 14.8 ms per loop (mean ± std. dev. of 300 runs, 1 loop each)


In [261]:
%timeit -n 1 -r 30 with torch.no_grad():ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.17 s ± 78.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [262]:
%timeit -n 1 -r 30 with torch.no_grad():modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

1.77 s ± 65.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# Torchscript

In [264]:
def workload_pnn_outer_torchscript(dim):
  print(f"now gen workload of pnn with config: dim: {dim}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1, method='outer')

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(22)],dim,[400,400,400],0.1, method='outer')
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_outer_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,10,None)),(slice(None,None,None),slice(10,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=22)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  ori = torch.jit.script(pnn_model_traced_ori)
  modify = torch.jit.script(pnn_model_traced_modify)
  ori.eval()
  modify.eval()
  return ori,modify

# dim = 8

In [288]:
ori, modify = workload_pnn_outer_torchscript(8)

now gen workload of pnn with config: dim: 8


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [289]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

12.3 ms ± 3.59 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [291]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

7.34 ms ± 1.78 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [268]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

36.4 ms ± 2.56 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [269]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

19.1 ms ± 2.35 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 32

In [270]:
ori, modify = workload_pnn_outer_torchscript(32)

now gen workload of pnn with config: dim: 32


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [271]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

106 ms ± 5.07 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [272]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

35.5 ms ± 3.02 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [273]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

452 ms ± 21.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [274]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

158 ms ± 7.02 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 64

In [275]:
ori, modify = workload_pnn_outer_torchscript(64)

now gen workload of pnn with config: dim: 64


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [276]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

448 ms ± 11.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [277]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

122 ms ± 14.3 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [278]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

1.59 s ± 68.5 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [279]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

472 ms ± 18.7 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


# dim = 128

In [280]:
ori, modify = workload_pnn_outer_torchscript(128)

now gen workload of pnn with config: dim: 128


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node redency_offset target redency_offset redency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/fx/graph.py:1460: UserWarning: Node unredency_offset target unredency_offset unredency_offset of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, 

In [281]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

1.51 s ± 53 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [282]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(1024,22), dtype=torch.long))

467 ms ± 14 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [283]:
%timeit -n 1 -r 30 with torch.no_grad(): output = ori(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

6.24 s ± 119 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [284]:
%timeit -n 1 -r 30 with torch.no_grad(): output = modify(torch.randint(low=0, high=88, size=(4096,22), dtype=torch.long))

1.75 s ± 64.1 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)
